In [1]:
class wraplist(object):
    def __init__(self, data):
        self.data = data
        self.n = len(data)
    def __getitem__(self, sl):
        if type(sl) == int:
            return self.data[sl]
        elif sl.start is not None and sl.start == sl.stop:
            return wraplist([])
        stop = None if sl.stop is None or sl.stop == self.n else sl.stop % self.n
        if stop is not None and sl.start >= stop:
            return wraplist(self.data[slice(sl.start, None, sl.step)] + self.data[slice(None, stop, sl.step)])
        else:
            return wraplist(self.data[slice(sl.start, stop, sl.step)])        
    def __setitem__(self, sl, val):
        if type(sl) == int:
            self.data[sl] = val
            return
        elif sl.start is not None and sl.start == sl.stop:
            return
        stop = None if sl.stop is None or sl.stop == self.n else sl.stop % self.n
        if stop is not None and sl.start >= stop:
            self.data[slice(sl.start, None, sl.step)] = val[slice(0, self.n - sl.start)]
            self.data[slice(None, stop, sl.step)] = val[slice(self.n - sl.start, None)]
            return
        else:
            self.data[slice(sl.start, stop, sl.step)] = val
            return
    def __len__(self):
        return len(self.data)
    def __repr__(self):
        return self.data.__repr__()

In [2]:
def revert(wl, start, length):
    #print(start, length, wl[start:start+length], wl[start:start+length][::-1])
    wl[start:start+length] = wl[start:start+length][::-1]

In [3]:
# test case
pos = 0
skip = 0
circle = wraplist(list(range(5)))
for l in list(map(int, "3, 4, 1, 5".split(', '))):
    revert(circle, pos, l)
    print(l, pos, circle)
    pos = (pos + l + skip) % len(circle)
    skip += 1
print(circle, circle[0] * circle[1])

3 0 [2, 1, 0, 3, 4]
4 3 [4, 3, 0, 1, 2]
1 3 [4, 3, 0, 1, 2]
5 1 [3, 4, 2, 1, 0]
[3, 4, 2, 1, 0] 12


In [4]:
def cycle(circle, lengths, pos, skip):
    for l in lengths:
        revert(circle, pos, l)
        # print(l, circle)
        pos = (pos + l + skip) % len(circle)
        skip += 1
    return circle, pos, skip

In [6]:
def densen(sparse_hash):
    dense_hash = []
    for i in range(16):
        offset = i * 16
        val = sparse_hash[offset]
        for j in range(1, 16):
            val ^= sparse_hash[offset + j]
        dense_hash.append(val)
    return dense_hash

def print_hash(dh):
    strings = map(lambda x: "{0:02x}".format(x), dh)
    return "".join(strings)

In [7]:
def getHash(string):
    inp = list(map(ord, string))
    inp += [17, 31, 73, 47, 23]
    circle = wraplist(list(range(256)))
    pos = 0
    skip = 0
    for i in range(64):
        circle, pos, skip = cycle(circle, inp, pos, skip)
    dh = densen(circle)
    return print_hash(dh)

In [8]:
assert(getHash('') == 'a2582a3a0e66e6e86e3812dcb672a272')
assert(getHash('AoC 2017') == '33efeb34ea91902bb2f59c9920caa6cd')
assert(getHash('1,2,3') == '3efbe78a8d82f29979031a4aa0b16a9d')
assert(getHash('1,2,4') == '63960835bcdc130f0b66d7ff4f6a5a8e')

In [40]:
def num_squares(inp_str):
    n = 0
    for i in range(128):
        string = inp_str + '-' + str(i)
        hash_str = getHash(string)
        hash_int = int(hash_str, base=16)
        while hash_int > 0:
            if hash_int % 2:
                n += 1
            hash_int >>= 1
    return n    

In [125]:
assert(num_squares('flqrgnkx') == 8108)

In [43]:
# part 1
print(num_squares('hfdlxzhv'))

8230


In [288]:
inp = 'hfdlxzhv'
# test
inp = 'flqrgnkx'
hashes = list(map(lambda x: getHash(inp + '-' + str(x)), range(128)))
ihashes = list(map(lambda x: int(x, base=16), hashes))
bhashes = list(map(lambda x: format(x, '#0130b')[2:], ihashes))

In [294]:
bhashes

['11010100111101110110101111011100101111111000001110001111100001000001011011001100111110101000101111000110110100011111100111100110',
 '01010101111010101011001111000100111110111111111011011110000101101101110011101100001011000110011011011101101000100110010001100100',
 '00001010110111110001001111111010010000001110100011101010100000010101001101110110011101110110101011110011101101111011001000110001',
 '10101101001111011010001010001100110101111011100011111011100110010111010000101100000011100110001101100111001011001010111101100010',
 '01101000001011111110010010001100010101011000011101101010101010101010000100011101111100100110001101001111100101101101001100011010',
 '11001001111101011111111111111100010001100100101001010110010111111001111010010100111100110010001110101000101010101010100101001011',
 '01000100110000000001101001110010011000100110110011101110000011100011101111000000111010001001100010001101000100110111010001100011',
 '1101011001110010001101100000100000110000110001100011100101001001010

In [405]:
from collections import defaultdict
import numpy as np
cluster = np.zeros((128, 128), dtype=np.int)
ncluster = 0
equal = defaultdict(set)

for i in range(128):
    for j in range(128):
        if bhashes[i][j] == '1':
            if cluster[i, j] == 0:
                ncluster += 1
                cluster[i, j] = ncluster
            if i < 127 and bhashes[i+1][j] == '1':
                if cluster[i+1, j] == 0 or cluster[i+1, j] == cluster[i, j]:
                    cluster[i+1, j] = cluster[i, j]
                else:
                    equal[cluster[i, j]].add(cluster[i+1, j]) 
                    cluster[i+1, j] = min(cluster[i, j], cluster[i+1, j])
            if j < 127 and bhashes[i][j+1] == '1':
                if cluster[i, j+1] == 0 or cluster[i, j+1] == cluster[i, j]:
                    cluster[i, j+1] = cluster[i, j]
                else:
                    equal[cluster[i, j]].add(cluster[i, j+1]) 
                    cluster[i, j+1] = min(cluster[i, j], cluster[i, j+1])
                

In [418]:
def gather(n, seen=set()):
    for el in equal[n]:
        if el not in seen and el in equal:
            seen.add(el)
            yield from gather(el, seen=seen)
        yield el


In [426]:
sorted(gather(5))

[8, 60, 74]

In [421]:
true_cluster = {}
for key in equal:
    tmp = sorted(set(gather(key)))
    if tmp[0] in true_cluster:
        true_cluster[tmp[0]] = true_cluster[tmp[0]].union(set(tmp[1:]))
    else:
        true_cluster[tmp[0]] = set(tmp[1:])

In [422]:
equal[4]

set()

In [423]:
true_cluster

{4: set(),
 5: set(),
 8: {60, 74},
 10: set(),
 11: set(),
 12: set(),
 13: {105, 287, 308},
 15: set(),
 16: set(),
 18: set(),
 19: set(),
 23: set(),
 24: set(),
 25: set(),
 28: set(),
 31: set(),
 38: set(),
 39: set(),
 41: set(),
 45: set(),
 47: set(),
 52: set(),
 53: set(),
 54: set(),
 57: {102, 115, 165, 204},
 59: set(),
 64: set(),
 66: set(),
 68: set(),
 72: set(),
 73: set(),
 77: {78},
 87: set(),
 88: set(),
 89: set(),
 91: set(),
 94: set(),
 95: set(),
 99: set(),
 107: set(),
 109: set(),
 113: set(),
 116: set(),
 119: set(),
 120: set(),
 121: set(),
 127: set(),
 132: set(),
 140: set(),
 141: set(),
 143: set(),
 149: set(),
 151: set(),
 153: set(),
 155: set(),
 156: set(),
 158: set(),
 170: set(),
 174: set(),
 175: {176, 226},
 177: set(),
 180: set(),
 185: set(),
 196: set(),
 197: set(),
 199: set(),
 200: set(),
 204: set(),
 206: set(),
 207: set(),
 208: set(),
 211: set(),
 212: set(),
 215: set(),
 216: set(),
 218: set(),
 220: set(),
 225: set

In [370]:
true_cluster = {}
for key in equal:
    tmp_cluster = sorted([key] + list(equal[key]))
    if tmp_cluster[0] in true_cluster:
        true_cluster[tmp_cluster[0]] = true_cluster[tmp_cluster[0]].union(set(tmp_cluster[1:]))
    else:
        true_cluster[tmp_cluster[0]] = set(tmp_cluster[1:])

In [383]:
def getK(n):
    for el in true_cluster[n]:
        if el in true_cluster:
            yield from getK(el)
        yield el
        

In [388]:
true_cluster

{4: {30},
 5: {8, 31, 60, 74, 117, 118},
 10: {11, 61},
 11: {12, 63},
 12: {13, 104, 105, 155, 191, 287, 307, 308, 345},
 13: {33, 47, 91},
 15: {16, 35},
 18: {66},
 19: {50, 67},
 23: {37, 38},
 24: {38},
 25: {39},
 28: {71},
 31: {45},
 39: {52},
 41: {53},
 45: {59},
 52: {68},
 53: {69, 111},
 54: {70},
 57: {73, 102, 115, 165, 204},
 59: {87},
 64: {77, 78, 93},
 66: {80},
 68: {82},
 72: {134},
 73: {86, 114, 188, 218, 219, 266, 338},
 87: {102},
 88: {120},
 89: {143},
 91: {105},
 94: {95, 106, 125},
 99: {112},
 107: {128},
 109: {130},
 113: {153},
 116: {135},
 119: {139, 221},
 120: {141},
 121: {145},
 127: {148},
 132: {150, 197, 198},
 140: {141, 154},
 142: {143},
 149: {160},
 151: {163},
 153: {164},
 155: {173},
 156: {175},
 158: {178, 227},
 168: {206},
 170: {207, 267, 324, 359},
 174: {175, 176, 192, 226, 243, 259, 289, 308},
 177: {193},
 180: {196},
 185: {201, 248, 249, 263, 300},
 186: {216},
 196: {213, 294},
 197: {230},
 199: {231},
 200: {215},
 204: {

In [393]:
sorted(getK(1372))

[1373,
 1387,
 1387,
 1400,
 1419,
 1420,
 1436,
 1436,
 1436,
 1451,
 1452,
 1453,
 1466,
 1467,
 1467,
 1468,
 1484,
 1485,
 1497,
 1532,
 1552,
 1573,
 1591]

In [ ]:
1372: {1373, 1387, 1400, 1420, 1436, 1452, 1467, 1485, 1552, 1573, 1591},
1373: {1387},

In [372]:
for key in true_cluster:
    for el in true_cluster[key]:
        if el < key:
            print(el, key)

In [373]:
len(true_cluster)

565

In [411]:
substract = 0
for key in true_cluster:
    substract += len(true_cluster[key])
print(ncluster, substract, ncluster - substract)

2152 511 1641


In [375]:
ncluster

2152

In [376]:
num_eq = 0
for key in equal:
    num_eq += len(equal[key])
print(ncluster, num_eq, ncluster - num_eq)

2152 936 1216


In [377]:
equal.pop?

In [378]:
cluster[:8, :8]

array([[  1,   1,   0,   2,   0,   3,   0,   0],
       [  0,   1,   0,   2,   0,   3,   0,  30],
       [  0,   0,   0,   0,  43,   0,  44,   0],
       [ 56,   0,  57,   0,  43,  43,   0,  58],
       [  0,  73,  57,   0,  43,   0,   0,   0],
       [ 86,  73,   0,   0,  43,   0,   0,  87],
       [  0,  73,   0,   0,   0, 102,   0,   0],
       [114,  73,   0, 115,   0, 102, 102,   0]])

In [ ]:
stack = 